In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Oct_dict_JSON.json', 'r') as json_file:
    link_day_minute_Oct_dict_JSON = json.load(json_file)

In [10]:
week_day_Oct_list = [1, 2, 3, 4, 5, 8, 9, 10, 11, 12, 15, 16, 17, 18, 19, 22, 23, 24, 25, 26, 29, 30, 31]

In [11]:
link_day_minute_Oct_list = []
for link_idx in range(24):
    for day in week_day_Oct_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Oct_list.append(link_day_minute_Oct_dict_JSON[key] ['AM_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Oct_list)

66240

In [13]:
x = np.matrix(link_day_minute_Oct_list)
x = np.matrix.reshape(x, 24, 2760)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
np.size(x, 0), np.size(x, 1)

(24, 2645)

In [15]:
x[:,:2]

matrix([[  1898.41437084,   2358.94402945],
        [  2311.64818463,   1711.86798692],
        [  2735.6127006 ,   3185.30655375],
        [  5786.79241704,   5359.24493481],
        [  2810.02443508,   2796.60924666],
        [ 11406.64962129,  10730.39715986],
        [  3169.99714535,   3074.04317923],
        [  3075.61861439,   3283.05739101],
        [  8094.32335225,   8440.08482888],
        [ 14281.49174789,  14730.6085683 ],
        [  7644.81146538,   6906.49513998],
        [ 11708.99130722,  11492.38607421],
        [ 12743.17963071,  12256.51773854],
        [  2895.07363898,   2756.34400449],
        [ 10725.87187658,  10160.51354753],
        [  2575.12070158,   2501.89087769],
        [  3685.1163859 ,   2966.22579833],
        [  1619.42115847,   2611.23387656],
        [ 15042.2830158 ,  14346.31836023],
        [  3705.63750748,   4745.47743477],
        [ 12472.50010794,  12396.33278345],
        [ 10144.27078917,  10010.51909352],
        [  4758.92801717,   6611

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b' * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [3e-01, 2e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -2.12242648e+06  0.00000000e+00  4.13e+03 1.27e+02  9.46e+05     0s
   1   6.95436131e+05 -1.81677751e+06  7.59e+02 1.71e+02  2.15e+05     0s
   2   7.46667841e+05 -1.39874312e+06  7.59e-04 5.68e-14  3.83e+04     0s
   3  -1.46310363e+04 -3.17133786e+05  7.60e-10 1.14e-13  5.4

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Oct_weekday_AM.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[6.642174565625127e-11,
 0.0024731300167046416,
 7.290918937694752e-11,
 0.002322913235717733,
 0.0024732224390318913,
 5398.188450852792,
 2.2001535979805912e-08,
 0.0012445065636211422,
 0.0024712540163281255,
 3117.647922471892,
 0.00234812392866153,
 0.0024720332743570217,
 598.8276359450766,
 0.0024632796962592296,
 128.86779323432836,
 5406.178495912746,
 138.67111608985994,
 0.6650009158966128,
 1221.4138866197743,
 1.7288162223675073e-06,
 7051.427999864956,
 1897.7696966761018,
 1770.3332993799825,
 1.25489322480734,
 3560.7827977780235,
 1.2559497106207544,
 5134.852131054291,
 4239.886949681056,
 872.4251272291059,
 0.7016504775880215,
 7510.559835662721,
 2140.3507070954374,
 7547.977966828832,
 2.01011972855882,
 0.0024671660746270613,
 5309.411292866887,
 0.7004730209584356,
 361.7714501223111,
 0.5944842992914996,
 2786.7050566949442,
 4950.01141352787,
 0.0024679506089255038,
 10.433301933808877,
 0.5887355184123706,
 5295.625690741101,
 3.83770962743672e-05,
 3.7919241